<a href="https://colab.research.google.com/github/jairathnishant/RAG_ChromaDB_llama3.2/blob/main/RAG_ChromaDB_llama3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pypdf[crypto]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 8.5 MB/s eta 0:00:00


In [2]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.2/159.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 5.9 MB/s eta 0:00:

In [3]:
!pip install sentence_transformers

In [4]:
!pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.1
    Uninstalling websockets-14.1:
      Successfully uninstalled websockets-14.1
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


## **Importing required packages**

In [5]:
# Libraries for loading and parsing the pdf book
from pypdf import PdfReader
import os

# For creating a database for storing embeddings
import chromadb
from chromadb.utils import embedding_functions

# For structuring the instructions to be passed to LLM
from transformers import pipeline

# For Retrieving & Storing the HF token
from huggingface_hub.hf_api import HfFolder
from google.colab import userdata

# For User Interface
import gradio as gr

# For response articulation via LLM
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

## **PDF parser**

In [6]:
def load_documents(file_path):

    """
    Loads the pdf file and returns the text.

    Parameters:
        file_path (string): file path as input of Biology pdf book

    Returns:
        string: chapter 4 & chapter 5 as a concatenated string
    """

    text = ""
    reader = PdfReader(file_path)
    number_of_pages = len(reader.pages)

    page_count = 0
    for page in range(number_of_pages):
        # took the page numbers via manual inspection
        if page_count >= 102 and page_count <= 146:
            page = reader.pages[page_count]
            text +=page.extract_text() + "/n"
        page_count += 1
    return text

file_path = (r'/content/ConceptsofBiology-WEB.pdf')

## **Creating chunks from the text corpus**

In [7]:
# Chunking of the document as per context window

def chunking(text: str, chunk_size: int = 500):

    """
    Chunk the corpus into smaller documents.

    Parameters:
        text (string): concatenated text of two chapters
        chunk_size (int): size of each chunk in words

    Returns:
        list: equal sized chunks of text
    """
    sentences = text.replace('\n', ' ').split('. ')
    chunks = []
    current_chunk = []
    current_size = 0

    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue

        # Ensure proper sentence ending
        if not sentence.endswith('.'):
            sentence += '.'

        sentence_size = len(sentence)

        # Check if adding this sentence would exceed chunk size
        if current_size + sentence_size > chunk_size and current_chunk:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
            current_size = sentence_size
        else:
            current_chunk.append(sentence)
            current_size += sentence_size

    # Add the last chunk if it exists
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

In [8]:
chunk = chunking(load_documents(file_path))
print(f'We have {len(chunk)} documents')

We have 239 documents


In [9]:
def process_document(file_path: str):

    """
    Process a single document and prepare it for ChromaDB.

    Parameters:
        file_path (string): file path as input of Biology pdf book

    Returns:
        int: id of the document
        string: text of the document
        dict: metadata of the document
    """

    # Read the document
    content = load_documents(file_path)

    # Split into chunks
    chunks = chunking(content)

    # Prepare metadata
    file_name = os.path.basename(file_path)
    metadatas = [{"source": file_name, "chunk": i} for i in range(len(chunks))]
    ids = [f"{file_name}_chunk_{i}" for i in range(len(chunks))]

    return ids, chunks, metadatas

## **Setting up vector database (ChromaDB) and storing text embeddings**

In [10]:
# Initialize ChromaDB client with persistence
client = chromadb.PersistentClient(path="chroma_db")

# Configure sentence transformer embeddings
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# Create or get existing collection
collection = client.get_or_create_collection(
    name="documents_collection",
    embedding_function=sentence_transformer_ef
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
def add_to_collection(collection, ids, texts, metadatas):

    """
    Add documents to ChromaDB in batches.

    Parameters:
        file_path (string): file path as input of Biology pdf book

    Returns:
        int: id of the document
        string: text of the document
        dict: metadata of the document
    """

    if not texts:
        return

    batch_size = 100
    for i in range(0, len(texts), batch_size):
        end_idx = min(i + batch_size, len(texts))
        collection.add(
            documents=texts[i:end_idx],
            metadatas=metadatas[i:end_idx],
            ids=ids[i:end_idx]
        )

In [12]:
# Calling functions to parse the pdf and then upload the data to ChromaDB

ids, texts, metadatas = process_document(file_path)
add_to_collection(collection, ids, texts, metadatas)

## **Carrying out Semantic search on the vector database**

In [32]:
def semantic_search(query: str, collection, n_results: int = 2):

    """
    Perform semantic search on the collection within ChromaDB.

    Parameters:
        collection: vector embeddings of the text corpus
        query (string): question asked by the user
        n_results (integer): top k semantic matches to be considered for answer

    Returns:
        dict: response generated comprising of text and metadata info
    """

    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )

    return results

def get_context_with_sources(results):

    """
    Extract context and source information from search results.

    Parameters:
        results (dict): response generated comprising of text and metadata info

    Returns:
        string: response collected by appending the top k matches
        dict: metadata info about the source name and chunk number of semantic match
    """

    # Combine document chunks into a single context
    context = "\n\n".join(results['documents'][0])

    # Format sources with metadata
    sources = [
        f"{meta['source']} (chunk {meta['chunk']})"
        for meta in results['metadatas'][0]
    ]

    return context, sources

## **Loading LLM from HF**

In [14]:
# Saving HF token via secret's feature of Colab

my_hf_key = userdata.get('RAG_llama3.2_proj')
HfFolder.save_token(my_hf_key)

In [15]:
# Storing LLM and tokenizer

model_name = "meta-llama/Llama-3.2-1B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

## **Prompt creation for LLM to articulate the final response**

In [16]:
def get_prompt(context: str, query: str):

    prompt_template =  f'''
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>{"You are given a context and a question. As a helpful assistant you need to answer question from the context below."}

    {context}

    {"Provide a conversational answer with brevity.If answer is not in the context, return 'I do not know'."}<|eot_id|>

    <|start_header_id|>user<|end_header_id|>{query}<|eot_id|>

    <|start_header_id|>assistant<|end_header_id|>'''

    return prompt_template

In [17]:
def generate_response(query: str, context: str):

    """
    Creat prompt using user query and semantic search for passing into LLM.

    Parameters:
        query (string): question asked by the user
        context (string): response collected by appending the top k matches

    Returns:
        dict: comprising of prompt (system instructions, user query) and  final response
    """

    prompt = get_prompt(context, query)
    # print(prompt)

    pipe = pipeline(
        "text-generation",
        model=model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        tokenizer = tokenizer,
        return_full_text = False
    )
    messages = [
                {"role": "user", "content": prompt}
            ],

    outputs = pipe(
        messages,
        max_new_tokens=4096,
        pad_token_id=tokenizer.eos_token_id
    )
    return(outputs[0][-1])

In [18]:
def rag_query(query: str, collection = collection, top_k: int = 4):

    """
    RAG query function to retrieve relevant chunks and generate answer.

    Parameters:
        collection: vector embeddings of the text corpus
        query (string): question asked by the user
        top_k (integer): top k semantic matches to be considered for answer

    Returns:
        string: final articulated response to the user query
        dict: list of source and respective chunk id from metadata
    """

    # Get relevant chunks
    results = semantic_search(query, collection, top_k)
    context, sources = get_context_with_sources(results)

    # Generate response
    response = generate_response(query, context)

    if response['generated_text'] == 'I do not know.':
        sources = ["Nil"] # getting jibberish in sources therefore adding this hot fix

    return response['generated_text'], sources

## **LLM based RAG Output**

In [20]:
query = "What is beta oxidation?"
response, sources = rag_query(query, collection)

# Print results
print("\nQuery:", query)
print("\nAnswer:", response)
print("\nSources used:")
for source in sources:
    print(f"- {source}")



Query: What is beta oxidation?

Answer: Beta oxidation is a key step in the citric acid cycle (also known as the Krebs cycle or TCA cycle). It's a process where acetyl-CoA, a molecule produced from the breakdown of carbohydrates, fats, and proteins, is converted into carbon dioxide and energy-rich molecules like ATP, NADH, and FADH2.

In beta oxidation, the acetyl-CoA molecule is broken down into acetyl-CoA units, releasing a molecule of carbon dioxide and a molecule of NADH and FADH2. These electrons are then passed through a series of reactions, eventually leading to the production of ATP, which is the primary energy currency of the cell.

Think of it like a chain reaction: 

- Acetyl-CoA → Acetyl group + CoA
- Acetyl group + NAD+ → NADH + H+ + CoA
- NADH + H+ → NAD+ + H2O

This process is essential for cellular energy production, as it allows the cell to generate energy from the breakdown of nutrients.

Sources used:
- ConceptsofBiology-WEB.pdf (chunk 88)
- ConceptsofBiology-WEB.pd

In [21]:
query = "Who is the president of India"
response, sources = rag_query(query, collection)

# Print results
print("\nQuery:", query)
print("\nAnswer:", response)
print("\nSources used:")
for source in sources:
    print(f"- {source}")


Query: Who is the president of India

Answer: I do not know.

Sources used:
- Nil


In [33]:
query = "What is the energy currency used by cells?"
response, sources = rag_query(query, collection)

# Print results
print("\nQuery:", query)
print("\nAnswer:", response)
print("\nSources used:")
for source in sources:
    print(f"- {source}")


Query: What is the energy currency used by cells?

Answer: The energy currency used by cells is called ATP (Adenosine Triphosphate).

Sources used:
- ConceptsofBiology-WEB.pdf (chunk 75)
- ConceptsofBiology-WEB.pdf (chunk 10)
- ConceptsofBiology-WEB.pdf (chunk 76)
- ConceptsofBiology-WEB.pdf (chunk 140)


## **Gradio - WIP**

In [31]:
# # Launching user interface using Gradio

# from functools import partial

# with gr.Blocks() as interface:
#     gr.Markdown("<center><h1>Biology Assistant")

#     query = gr.Textbox(label = "Please type your question here -", lines = 3)

#     submit_btn = gr.Button(variant = "Primary", size = 'sm')

#     with gr.Row() as output_row:
#             llama_output = gr.Markdown("Response")

#     gr.on(
#         triggers = [query.submit, submit_btn.click],
#         fn = partial(rag_query, collection=collection),
#         inputs = query,
#         outputs = llama_output,
#         show_progress = "hidden"
# )

# interface.launch(debug = True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a5a8bf30271c97fbc7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2015, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1562, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a5a8bf30271c97fbc7.gradio.live
